# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel30', 'models','t_dGP_train')

Linesearch     25;  Value 2.862214e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

In [11]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 2.12480201, MSLL -0.2019, NLPD 2.6564


### 1.1.4 Find multi q

In [12]:
criterion = 'TERBCM';

In [6]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.2048
   2.2027
   2.2002
   2.2032
   2.2014
   2.2062
   2.2022
   2.2094
   2.2048
   2.2090
   2.2060
   2.2065
   2.2033
   2.2041
   2.2019
   2.2096
   2.2030
   2.2094
   2.2052
   2.2061
   2.2031
   2.2031
   2.2091
   2.2060
   2.2020
   2.2023
   2.2076
   2.2008
   2.2085
   2.2047
   2.2075
   2.2066
   2.2014
   2.2080
   2.2099
   2.2056
   2.2003
   2.2035
   2.2078
   2.2090
   2.2030
   2.2045
   2.2092
   2.2010
   2.2052
   2.2027
   2.2053
   2.2098
   2.2066
   2.2059

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finish preprocessing...
qs =

   2.5633
   2.5731
   2.

In [13]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.40139
   0.40799
   0.40848
   0.40121
   0.40822
   0.40008
   0.40352
   0.40971
   0.40684
   0.40275
   0.40708
   0.40339
   0.40033
   0.40568
   0.40333
   0.40340
   0.40305
   0.40770
   0.40594
   0.40378
   0.40317
   0.40174
   0.40499
   0.40157
   0.40261
   0.40871
   0.40438
   0.40522
   0.40712
   0.40247
   0.40780
   0.40056
   0.40363
   0.40518
   0.40496
   0.40358
   0.40350
   0.40214
   0.40028
   0.40368
   0.40087
   0.40311
   0.40830
   0.40472
   0.40829
   0.40842
   0.40663
   0.40477
   0.40533
   0.40014

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finis

## 1.2 GRBCM vs TEGRBCM

In [14]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 2.10349598, MSLL -1.3020, NLPD 1.5563


### 1.2.4 Find multi q

In [15]:
criterion = 'TEGRBCM';

In [8]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0072
   2.0047
   2.0029
   2.0015
   2.0051
   2.0066
   2.0050
   2.0045
   2.0066
   2.0037
   2.0082
   2.0083
   2.0022
   2.0070
   2.0060
   2.0038
   2.0062
   2.0037
   2.0091
   2.0043
   2.0013
   2.0009
   2.0043
   2.0037
   2.0095
   2.0062
   2.0097
   2.0090
   2.0058
   2.0034
   2.0089
   2.0035
   2.0090
   2.0004
   2.0099
   2.0022
   2.0063
   2.0025
   2.0044
   2.0092
   2.0042
   2.0069
   2.0087
   2.0061
   2.0041
   2.0013
   2.0071
   2.0041
   2.0013
   2.0076

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finish preprocessing...
qs =

   2.0072
   2.0047
   2.

In [16]:
printf('Optimizing q\r\n');
iqs = 0.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.20877
   0.20852
   0.20492
   0.20949
   0.20218
   0.20141
   0.20382
   0.20729
   0.20654
   0.20307
   0.20709
   0.20315
   0.20045
   0.20471
   0.20303
   0.20608
   0.20245
   0.20797
   0.20286
   0.20827
   0.20392
   0.20383
   0.20784
   0.20399
   0.20586
   0.20298
   0.20350
   0.20164
   0.20024
   0.20716
   0.20232
   0.20370
   0.20641
   0.20723
   0.20223
   0.20049
   0.20272
   0.20861
   0.20806
   0.20627
   0.20091
   0.20914
   0.20630
   0.20610
   0.20717
   0.20537
   0.20198
   0.20972
   0.20859
   0.20014

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finis

## 1.3 GPoE vs TEGPoE

In [17]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 1.64280822, MSLL -0.1205, NLPD 2.7378


### 1.3.4 Find multi q

In [18]:
criterion = 'TEGPoE';

In [10]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0057
   2.0062
   2.0058
   2.0015
   2.0088
   2.0050
   2.0068
   2.0033
   2.0022
   2.0062
   2.0027
   2.0081
   2.0007
   2.0086
   2.0064
   2.0012
   2.0086
   2.0017
   2.0095
   2.0014
   2.0021
   2.0021
   2.0005
   2.0041
   2.0007
   2.0014
   2.0084
   2.0097
   2.0076
   2.0030
   2.0056
   2.0068
   2.0028
   2.0082
   2.0005
   2.0042
   2.0041
   2.0068
   2.0064
   2.0037
   2.0043
   2.0069
   2.0060
   2.0079
   2.0075
   2.0055
   2.0015
   2.0056
   2.0070
   2.0097

ld =  10
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finish preprocessing...
qs =

   2.1114
   2.1234
   2.0

In [19]:
printf('Optimizing q\r\n');
iqs = 1.8 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   1.8065
   1.8004
   1.8024
   1.8034
   1.8077
   1.8028
   1.8046
   1.8017
   1.8010
   1.8060
   1.8029
   1.8090
   1.8061
   1.8028
   1.8016
   1.8018
   1.8017
   1.8014
   1.8017
   1.8031
   1.8073
   1.8051
   1.8016
   1.8064
   1.8098
   1.8062
   1.8040
   1.8044
   1.8084
   1.8051
   1.8095
   1.8045
   1.8051
   1.8033
   1.8020
   1.8011
   1.8041
   1.8022
   1.8005
   1.8031
   1.8019
   1.8028
   1.8077
   1.8003
   1.8022
   1.8002
   1.8073
   1.8002
   1.8051
   1.8023

ld =  10
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
k =  33
k =  34
k =  35
k =  36
k =  37
k =  38
k =  39
k =  40
k =  41
k =  42
k =  43
k =  44
k =  45
k =  46
k =  47
k =  48
k =  49
k =  50
Finish preprocessing...
qs =

   1.9830
   1.9887
   1.8